<a href="https://colab.research.google.com/github/BhaveshWadibhasme/Image-Segmentation/blob/main/new_segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import glob
import numpy as np
import cv2
from PIL import Image

!pip install git+https://github.com/qubvel/segmentation_models


In [ ]:
input_dir = "road_segmentation_ideal/train_data/"
target_dir = "road_segmentation_ideal/test_data"

In [ ]:
y = np.zeros((100, 256,256), dtype=np.float32)
X = np.zeros((100,256, 256, 1), dtype=np.float32)


    

In [ ]:
train_imgs = glob.glob("road_segmentation_ideal/training/input/*png")
for i in range(len(train_imgs)):
    if i>=100:
        break
    img = Image.open(train_imgs[i])
    img = img.convert('L')
    img = img.resize((256,256))
    img = np.array(img).reshape(256,256,1)
    X[i] = img/255

In [ ]:
tar_imgs = glob.glob("road_segmentation_ideal/training/output/*png")
for i in range(len(tar_imgs)):
    if i>=100:
        break
    mask_img = Image.open(tar_imgs[i])
    mask_img = mask_img.convert('L')
    mask_img = mask_img.resize((256,256))
    mask_img = np.array(mask_img)/255
    mask_img = (mask_img>0.5)*1.0  
    y[i] = mask_img
    
    

In [ ]:
X.shape,y.shape

((100, 256, 256, 1), (100, 256, 256))

In [ ]:
from segmentation_models import Unet
#from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from keras.models import model_from_json

from keras.layers import Input, Conv2D, Reshape
from keras.models import Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
BACKBONE = 'resnet34'
#preprocess_input = get_preprocessing(BACKBONE)
x_train = X_train
x_test = X_test

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((80, 256, 256, 1), (20, 256, 256, 1), (80, 256, 256), (20, 256, 256))

In [ ]:
np.unique(y_test[0])

array([0., 1.], dtype=float32)

In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow as tf
from segmentation_models import Unet
from tensorflow.keras.optimizers import Adam
from segmentation_models.losses import bce_jaccard_loss, bce_dice_loss
from segmentation_models.metrics import iou_score
from sklearn.pipeline import Pipeline
from keras.callbacks import EarlyStopping

In [ ]:
def unet(input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs,conv10)

    model.compile(optimizer ="Adam", loss = bce_jaccard_loss, metrics = iou_score)
    
    print(model.summary())

 

    return model


In [ ]:
model = unet()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 256, 256, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_25[0][0]                  
____________________________________________________________________________________________

In [ ]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='max')
callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0),checkpoint]

history=model.fit(X_train,y_train,batch_size =32,epochs=100,validation_data=(X_test, y_test))

Epoch 1/100
3/3 [==============================] - 151s 47s/step - loss: 1.5769 - iou_score: 1.6650e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 2/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_score: 1.6492e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 3/100
3/3 [==============================] - 149s 46s/step - loss: 1.5769 - iou_score: 1.7521e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 4/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_score: 1.6235e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 5/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_score: 1.6724e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 6/100
3/3 [==============================] - 147s 46s/step - loss: 1.5769 - iou_score: 1.5814e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 7/100
3/3 [==============================] - 147s 46s/step - loss: 1.5769 - iou_score: 1

Epoch 55/100
3/3 [==============================] - 147s 46s/step - loss: 1.5769 - iou_score: 1.7183e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 56/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_score: 1.9388e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 57/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_score: 1.7792e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 58/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_score: 1.6194e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 59/100
3/3 [==============================] - 147s 46s/step - loss: 1.5769 - iou_score: 1.5988e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 60/100
3/3 [==============================] - 147s 46s/step - loss: 1.5769 - iou_score: 1.9043e-10 - val_loss: 1.5535 - val_iou_score: 2.1260e-10
Epoch 61/100
3/3 [==============================] - 148s 46s/step - loss: 1.5769 - iou_s

In [ ]:
pred_mask=[]
for i in range(len(X_test)):
    mask=model.predict(X_test[i].reshape(1,256,256,1))
    mask=mask.reshape(256,256)
    mask=mask.flatten()
    mask=[ 1 if x >= 0.2 else 0 for x in mask]
    pred_mask.extend(mask)
    
    

In [ ]:
label_mask=y_test.flatten()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
len(label_mask),len(pred_mask)

(1310720, 1310720)

In [ ]:
print(classification_report(pred_mask,label_mask))

              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98   1310720
         1.0       0.00      0.00      0.00         0

    accuracy                           0.96   1310720
   macro avg       0.50      0.48      0.49   1310720
weighted avg       1.00      0.96      0.98   1310720

